In [20]:
from sklearn.utils import shuffle
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm
import numpy as np
import pickle
import os
import pandas as pd
import scipy
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance
from sklearn.datasets import make_blobs
import math
import argparse
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from ucimlrepo import fetch_ucirepo 
from collections import Counter
from scipy.stats import entropy
from sklearn import tree
from sklearn import preprocessing
from itertools import product
from algs_lib import *
import sys
import imblearn

In [21]:
train_x, train_y, test_x, test_y, num_classes, train_len = gen_bean(normalize=True)


In [51]:
def run_kmeans(train_x, train_y, num_clusters, seed, weights = None):
    rand_state = np.random.RandomState(seed)
    sm = SMOTE(random_state=seed)
    train_x, train_y = sm.fit_resample(train_x, train_y)
    model = KMeans(n_clusters=num_clusters, random_state=rand_state,
                   max_iter = 1000, init='k-means++', n_init=10).fit(train_x)
    centers = model.cluster_centers_

    assert len(centers) == num_clusters
    mapping = infer_cluster_labels(model, train_y)

    ordered_centers = []
    new_centers = []
    
    for ind in range(len(centers)):
        index = mapping[ind]
        curr_center = centers[index]
        new_centers.append(curr_center)
        for k in curr_center:
            ordered_centers.append(k)
    model.cluster_centers_ = np.array(new_centers)
    return model, ordered_centers

# def run_kmeans(train_x, train_y, num_clusters, seed):
#     rand_state = np.random.RandomState(seed)
#     sm = SMOTE(random_state=seed)
#     train_x, train_y = sm.fit_resample(train_x, train_y)
#     model = KMeans(n_clusters=num_clusters, random_state=rand_state,
#         n_init="auto", init='k-means++').fit(train_x)
    
#     centers = model.cluster_centers_
#     assert len(centers) == num_clusters
#     mapping = infer_cluster_labels(model, train_y)
    
#     ordered_centers = []
#     new_centers = []
    
#     for ind in range(len(centers)):
#         index = mapping[ind]
#         curr_center = centers[index]
#         new_centers.append(curr_center)
#         for k in curr_center:
#             ordered_centers.append(k)
#     model.cluster_centers_ = np.array(new_centers)
#     return model, ordered_centers

In [52]:
from imblearn.over_sampling import SMOTE


In [53]:
sm = SMOTE(random_state = 65703)
X_res, y_res = sm.fit_resample(train_x, train_y)

In [54]:
model, cluster_centers = run_kmeans(X_res, y_res, num_clusters=num_classes, seed=None)
predictions = model.predict(test_x)
acc = accuracy_score(test_y, predictions)
print(acc)

0.8577026696056821


In [57]:
from sklearn.utils import class_weight
num_trials = 250
subsample_rate = int(0.5*train_len)

# K MEANS
avg_acc = 0
for i in range(num_trials):
    shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
    shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
#     sm = SMOTE()
#     X_res, y_res = sm.fit_resample(shuffled_x1, shuffled_y1)
    model, cluster_centers = run_kmeans(X_res, y_res, num_clusters=num_classes, seed=i)
    predictions = model.predict(test_x)
    acc = accuracy_score(test_y, predictions)
    print(f'i={i}, acc is {acc}')
    avg_acc += acc
avg_acc /= num_trials

# 1/

i=0, acc is 0.8545187362233652
i=1, acc is 0.8540289003183933
i=2, acc is 0.855008572128337
i=3, acc is 0.855008572128337
i=4, acc is 0.855008572128337
i=5, acc is 0.8540289003183933
i=6, acc is 0.8547636541758511
i=7, acc is 0.855008572128337
i=8, acc is 0.603722752877786
i=9, acc is 0.8547636541758511
i=10, acc is 0.8540289003183933
i=11, acc is 0.8547636541758511
i=12, acc is 0.8540289003183933
i=13, acc is 0.8545187362233652
i=14, acc is 0.8545187362233652
i=15, acc is 0.8547636541758511
i=16, acc is 0.8545187362233652
i=17, acc is 0.8547636541758511
i=18, acc is 0.8540289003183933
i=19, acc is 0.8547636541758511
i=20, acc is 0.8545187362233652
i=21, acc is 0.855008572128337
i=22, acc is 0.8547636541758511
i=23, acc is 0.8547636541758511
i=24, acc is 0.8557433259857947
i=25, acc is 0.855008572128337
i=26, acc is 0.8547636541758511
i=27, acc is 0.8545187362233652
i=28, acc is 0.8547636541758511
i=29, acc is 0.4937545922116091
i=30, acc is 0.8545187362233652
i=31, acc is 0.8550085721

In [56]:
avg_acc

0.831754102375704

In [31]:
from sklearn.cluster import KMeans


In [33]:
KMeans(n_clusters=10, random_state=None,
                   max_iter = 1000, n_init=10, return_n_iter=False)

TypeError: __init__() got an unexpected keyword argument 'return_n_iter'

In [34]:
from sklearn.cluster import kmeans_plusplus

In [36]:
centers, indices = kmeans_plusplus(train_x, n_clusters=2, random_state=0)

In [37]:
centers

array([[0.03888196, 0.08302148, 0.09594299, 0.1110889 , 0.3239974 ,
        0.74853938, 0.03835895, 0.07999182, 0.80444649, 0.86588474,
        0.79725713, 0.51326398, 0.68349399, 0.53449123, 0.46005332,
        0.84681744],
       [0.16963142, 0.29628161, 0.36181116, 0.23192067, 0.63303858,
        0.91527671, 0.1652836 , 0.28297299, 0.69180984, 0.93292515,
        0.66818607, 0.22826845, 0.47107852, 0.15928971, 0.19074727,
        0.84658251]])

In [41]:
indices

array([2732, 5578])

In [19]:
len(shuffled_y1)

9527

In [30]:
weights = []
for i in range(7):
    weights.append(1 / (len([ind for ind in range(len(train_y)) if train_y[ind] == i]) / train_len))

In [31]:
weights

[5.144168466522679,
 6.964181286549708,
 25.203703703703702,
 3.8477382875605817,
 6.809864188706219,
 10.446271929824562,
 8.342381786339756]